# load_bronze_to_silver

Este notebook se encarga de realizar la limpieza de los datos de bronze, incluyendo:

- Nulos
- Valores atípicos (outliers)
- Duplicados
- Tipado

## Imports

In [1]:
import os

import pandas as pd

## Parametros

In [3]:
bronze_path = os.path.join(
    os.curdir, "Data_Lake", "Bronze"
)
silver_path = os.path.join(
    os.curdir, "Data_Lake", "Silver"
)

In [113]:
bronze_file_name = "erp_bronze.parquet"

silver_file_name = "erp_silver.parquet"

In [114]:
bronze_file_path = os.path.join(
    bronze_path, bronze_file_name
)
silver_file_path = os.path.join(
    silver_path, silver_file_name
)

## Main

### Cargar los datos de bronze

In [7]:
bronze_erp_df = pd.read_parquet(bronze_file_path)

In [8]:
bronze_erp_df.head()

,id_cliente,nombre,email,edad,ciudad,salario,fecha_registro,categoria,telefono,activo,puntos_fidelidad,ultima_compra,_BronzeTimestamp
0,1,Cliente_1,cliente1@email.com,66.0,Valencia,2235.894632,2016-03-22 22:25:26.825944,B,+34 674293979,False,NaN,2025-04-02 22:25:26.833943,2025-11-03 19:15:59.714757
1,2,Cliente_2,cliente2@email.com,None,Barcelona,44151.957628,2025-10-08 22:25:26.825944,E,+34 663639417,1,NaN,2024-12-23 22:25:26.833943,2025-11-03 19:15:59.714757
2,3,Cliente_3,cliente3@email.com,75.0,Bilbao,35136.416478,2017-05-12 22:25:26.825944,C,+34 652916829,True,NaN,2025-09-25 22:25:26.833943,2025-11-03 19:15:59.714757
3,4,Cliente_4,cliente4@email.com,35.0,Bilbao,13325.620084,2021-12-16 22:25:26.825944,C,+34 641535944,True,200.0,2025-02-17 22:25:26.833943,2025-11-03 19:15:59.714757
4,5,Cliente_5,cliente5@email.com,38.0,Valencia,42037.063235,2018-11-05 22:25:26.825944,C,+34 602269798,True,200.0,2023-11-18 22:25:26.833943,2025-11-03 19:15:59.714757


###  Tratar Nulos

De cara a tratar los valores nulos, lo primero que debemos comprobar es que campos son críticos para la tabla.

Si tenemos un "id" que usaremos para identificar a una fila de manera única, no puede ser nulo.

De ahí en adelante debemos usar nuestro propio juicio para decidir que puede ser nulo y que no, y actuar en consecuencia:

- Eliminando las filas con valores nulos en esa columna
- Imputando los valores que faltan

In [ ]:
# Compruebo si la columna de id contiene nulos
bronze_erp_df[bronze_erp_df['id_cliente'].isnull() == True]

,id_cliente,nombre,email,edad,ciudad,salario,fecha_registro,categoria,telefono,activo,puntos_fidelidad,ultima_compra,_BronzeTimestamp


In [ ]:
# Compruebo el resto de columnas
bronze_erp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id_cliente        1020 non-null   int64         
 1   nombre            970 non-null    object        
 2   email             1020 non-null   object        
 3   edad              948 non-null    object        
 4   ciudad            895 non-null    object        
 5   salario           1020 non-null   float64       
 6   fecha_registro    1020 non-null   object        
 7   categoria         1020 non-null   object        
 8   telefono          1020 non-null   object        
 9   activo            1020 non-null   object        
 10  puntos_fidelidad  703 non-null    float64       
 11  ultima_compra     1020 non-null   object        
 12  _BronzeTimestamp  1020 non-null   datetime64[us]
dtypes: datetime64[us](1), float64(2), int64(1), object(9)
memory usage: 103.7+ KB


In [ ]:
# Defino como voy a tratar los nulos en el resto de columnas
fill_values = {
    "puntos_fidelidad": bronze_erp_df['puntos_fidelidad'].mean(),
    "nombre": "Desconocido",
    "ciudad": "Desconocido"
}
# NOTE: En edad, dejaremos los nulos como estan

In [ ]:
# Aplico la limpieza
bronze_erp_clean_df = bronze_erp_df.fillna(fill_values)

In [ ]:
# Valido que se han corregido correctamente
bronze_erp_clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id_cliente        1020 non-null   int64         
 1   nombre            1020 non-null   object        
 2   email             1020 non-null   object        
 3   edad              948 non-null    object        
 4   ciudad            1020 non-null   object        
 5   salario           1020 non-null   float64       
 6   fecha_registro    1020 non-null   object        
 7   categoria         1020 non-null   object        
 8   telefono          1020 non-null   object        
 9   activo            1020 non-null   object        
 10  puntos_fidelidad  1020 non-null   float64       
 11  ultima_compra     1020 non-null   object        
 12  _BronzeTimestamp  1020 non-null   datetime64[us]
dtypes: datetime64[us](1), float64(2), int64(1), object(9)
memory usage: 103.7+ KB


### Valores atipicos (outliers)

Pese a que no se trata de errores en el dataset (como los nulos o los duplicados) los valores atípicos son registros en alguna columna que no tienen sentido lógico, como un valor para edad de un humano de 300 o -5.

No hay una regla fuera de la lógica básica para detectar estos valores, pero hay distintas maneras de comprobarlos.

In [ ]:
bronze_erp_clean_df['email'].value_counts(dropna=False)
# NOTE: Aquí podemos ver que tenemos 101 filas con un valor de "correo_no_valido"
# en la columna email

email
correo_no_valido        101
cliente211@email.com      2
cliente205@email.com      2
cliente152@email.com      2
cliente283@email.com      2
                       ... 
cliente996@email.com      1
cliente997@email.com      1
cliente998@email.com      1
cliente999@email.com      1
cliente4@email.com        1
Name: count, Length: 901, dtype: int64

In [ ]:
bronze_erp_clean_df['edad'].value_counts(dropna=False)
# NOTE: En el caso de edad, podemos ver que no solo tenemos nulos, sino
# que no todos los valores son un numero, debemos corregirlo.

edad
veinticinco    102
None            72
15.0            22
52.0            20
47.0            20
              ... 
79.0             8
24.0             8
25.0             8
57.0             7
45.0             6
Name: count, Length: 67, dtype: int64

In [35]:
bronze_erp_clean_df['edad'] = bronze_erp_clean_df['edad'].replace("veinticinco",25)

In [ ]:
# NOTE: Una vez solucionado el error de tipo, comprobamos los extremos del dataset
sorted(bronze_erp_clean_df['edad'].value_counts(dropna=False))

[6,
 7,
 8,
 8,
 8,
 8,
 8,
 9,
 9,
 9,
 10,
 10,
 10,
 10,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 14,
 14,
 14,
 14,
 14,
 14,
 15,
 15,
 15,
 15,
 15,
 15,
 16,
 16,
 16,
 16,
 16,
 16,
 17,
 18,
 18,
 18,
 19,
 20,
 20,
 22,
 72,
 102]

In [ ]:
# NOTE: Comprobamos si el tipo de los datos es correcto, y si no lo
# cambiamos
bronze_erp_clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id_cliente        1020 non-null   int64         
 1   nombre            1020 non-null   object        
 2   email             1020 non-null   object        
 3   edad              948 non-null    object        
 4   ciudad            1020 non-null   object        
 5   salario           1020 non-null   float64       
 6   fecha_registro    1020 non-null   object        
 7   categoria         1020 non-null   object        
 8   telefono          1020 non-null   object        
 9   activo            1020 non-null   object        
 10  puntos_fidelidad  1020 non-null   float64       
 11  ultima_compra     1020 non-null   object        
 12  _BronzeTimestamp  1020 non-null   datetime64[us]
dtypes: datetime64[us](1), float64(2), int64(1), object(9)
memory usage: 103.7+ KB


In [47]:
bronze_erp_clean_df['edad'] = bronze_erp_clean_df['edad'].astype("float")

In [48]:
bronze_erp_clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id_cliente        1020 non-null   int64         
 1   nombre            1020 non-null   object        
 2   email             1020 non-null   object        
 3   edad              948 non-null    float64       
 4   ciudad            1020 non-null   object        
 5   salario           1020 non-null   float64       
 6   fecha_registro    1020 non-null   object        
 7   categoria         1020 non-null   object        
 8   telefono          1020 non-null   object        
 9   activo            1020 non-null   object        
 10  puntos_fidelidad  1020 non-null   float64       
 11  ultima_compra     1020 non-null   object        
 12  _BronzeTimestamp  1020 non-null   datetime64[us]
dtypes: datetime64[us](1), float64(3), int64(1), object(8)
memory usage: 103.7+ KB


In [ ]:
bronze_erp_clean_df['fecha_registro'].value_counts(dropna=False)
# NOTE: Aqui podemos ver que existen fechas inválidas (mes 13 dia 45)
# y que los timestamp (hora) son iguales

fecha_registro
2023-13-45                    51
2024-04-22 22:25:26.828945     3
2018-03-29 22:25:26.827944     2
2017-01-31 22:25:26.828945     2
2019-01-05 22:25:26.825944     2
                              ..
2024-05-20 22:25:26.831943     1
2020-05-29 22:25:26.831943     1
2025-03-27 22:25:26.831943     1
2024-02-07 22:25:26.831943     1
2019-06-08 22:25:26.831943     1
Name: count, Length: 930, dtype: int64

In [53]:
# Limpiar fechas erroneas
bronze_erp_dates_df = bronze_erp_clean_df[
    bronze_erp_clean_df['fecha_registro'] != '2023-13-45']

In [55]:
# Limpiar timestamp y convertir a fecha
bronze_erp_dates_df['fecha_registro'] = pd.to_datetime(bronze_erp_dates_df['fecha_registro']).dt.date

C:\Users\Alumno\AppData\Local\Temp\ipykernel_9936\593484380.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bronze_erp_dates_df['fecha_registro'] = pd.to_datetime(bronze_erp_dates_df['fecha_registro']).dt.date


In [ ]:
sorted(bronze_erp_dates_df['salario'])
# NOTE: Aqui vemos que tenemos salarios negativos

[-49747.0817562402,
 -25733.798108818195,
 -20826.4212398459,
 -18980.932957932066,
 -18055.2833632101,
 -12093.256642478147,
 -11393.960989093262,
 -7852.312053118163,
 -7761.496535962935,
 -7527.0429208179485,
 -7028.143738264625,
 -6786.103851483938,
 -6475.703568932222,
 -6324.245930707024,
 -6246.776478187348,
 -5765.350130344319,
 -5242.924604877982,
 -5033.590862499419,
 -4153.690471494119,
 -4078.288746389029,
 -3060.5636287685998,
 -2607.3188435204647,
 -2484.678426061855,
 -2354.7096704950964,
 -2341.5154706222493,
 -1996.995526057657,
 -1468.431619630108,
 387.93827496169615,
 733.2228772671442,
 1127.8248305600027,
 1617.2362676449957,
 1682.62741124471,
 1976.9940006728284,
 2162.4334357451407,
 2235.89463157098,
 2671.429010214688,
 2776.9043343866324,
 2915.9179063985066,
 3320.5375018529485,
 3410.8480752626747,
 3490.5397033413574,
 3885.272139474389,
 3927.292914951846,
 4557.313984566186,
 4883.33604258456,
 5111.330724162744,
 5263.76499098666,
 5527.487679237529,
 

In [65]:
bronze_erp_salary_df = bronze_erp_dates_df

In [ ]:
bronze_erp_salary_df['salario'] = bronze_erp_dates_df['salario'].abs()
# NOTE: Decidimos corregirlo usando el valor absoluto de salario, aunque
# se puede usar cualquier otro método mientras tenga sentido

C:\Users\Alumno\AppData\Local\Temp\ipykernel_9936\2599068689.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bronze_erp_salary_df['salario'] = bronze_erp_dates_df['salario'].abs()


In [67]:
sorted(bronze_erp_salary_df['salario'])

[387.93827496169615,
 733.2228772671442,
 1127.8248305600027,
 1468.431619630108,
 1617.2362676449957,
 1682.62741124471,
 1976.9940006728284,
 1996.995526057657,
 2162.4334357451407,
 2235.89463157098,
 2341.5154706222493,
 2354.7096704950964,
 2484.678426061855,
 2607.3188435204647,
 2671.429010214688,
 2776.9043343866324,
 2915.9179063985066,
 3060.5636287685998,
 3320.5375018529485,
 3410.8480752626747,
 3490.5397033413574,
 3885.272139474389,
 3927.292914951846,
 4078.288746389029,
 4153.690471494119,
 4557.313984566186,
 4883.33604258456,
 5033.590862499419,
 5111.330724162744,
 5242.924604877982,
 5263.76499098666,
 5527.487679237529,
 5539.899488771036,
 5609.833623351704,
 5765.350130344319,
 5837.333166573801,
 6034.321203398868,
 6110.983734845871,
 6246.776478187348,
 6324.245930707024,
 6475.703568932222,
 6514.921904576739,
 6560.178221008824,
 6650.76043804795,
 6659.899008394754,
 6714.338076224078,
 6768.556168150819,
 6786.103851483938,
 6924.450015478451,
 7000.88094

In [ ]:
bronze_erp_salary_df.info() # Comprobamos el tipo

<class 'pandas.core.frame.DataFrame'>
Index: 969 entries, 0 to 1019
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id_cliente        969 non-null    int64         
 1   nombre            969 non-null    object        
 2   email             969 non-null    object        
 3   edad              902 non-null    float64       
 4   ciudad            969 non-null    object        
 5   salario           969 non-null    float64       
 6   fecha_registro    969 non-null    object        
 7   categoria         969 non-null    object        
 8   telefono          969 non-null    object        
 9   activo            969 non-null    object        
 10  puntos_fidelidad  969 non-null    float64       
 11  ultima_compra     969 non-null    object        
 12  _BronzeTimestamp  969 non-null    datetime64[us]
dtypes: datetime64[us](1), float64(3), int64(1), object(8)
memory usage: 106.0+ KB


In [ ]:
bronze_erp_salary_df['ultima_compra'].value_counts(dropna=False)
# NOTE: No todos los valores en esta columna de timestamps son timestamps

ultima_compra
No registrada                 47
2025-04-03 22:25:26.835945     4
2025-10-08 22:25:26.834944     3
2023-11-14 22:25:26.834944     3
2024-10-06 22:25:26.834944     3
                              ..
2024-03-11 22:25:26.836945     1
2024-12-06 22:25:26.836945     1
2024-06-18 22:25:26.836945     1
2025-09-05 22:25:26.833943     1
2024-12-23 22:25:26.833943     1
Name: count, Length: 772, dtype: int64

In [70]:
bronze_erp_ultima_compra_df = bronze_erp_salary_df

In [ ]:
# NOTE: Decidimos corregirlo usando la fecha de registro como fecha de ultima_compra
# para los valores inválidos

In [74]:
def reemplazar_fechas_erroneas(fila: dict)->None:
    if fila['ultima_compra'] == "No registrada":
        return fila['fecha_registro']
    else:
        return fila['ultima_compra']

In [ ]:
bronze_erp_ultima_compra_df = bronze_erp_ultima_compra_df.apply(reemplazar_fechas_erroneas, axis=1)

In [76]:
bronze_erp_ultima_compra_df['ultima_compra'].value_counts(dropna=False)

ultima_compra
2025-04-03 22:25:26.835945    4
2024-01-24 22:25:26.835945    3
2025-10-08 22:25:26.834944    3
2024-05-10 22:25:26.834944    3
2025-02-21 22:25:26.834944    3
                             ..
2024-03-11 22:25:26.836945    1
2024-12-06 22:25:26.836945    1
2024-06-18 22:25:26.836945    1
2025-09-05 22:25:26.833943    1
2025-07-12 22:25:26.836945    1
Name: count, Length: 818, dtype: int64

In [77]:
bronze_erp_category =bronze_erp_ultima_compra_df

In [81]:
bronze_erp_category['categoria'].value_counts(dropna=False)

categoria
INVALID    146
G          126
F          125
C          124
D          121
E          118
B          110
A           99
Name: count, dtype: int64

In [82]:
bronze_erp_activo_df = bronze_erp_category

In [ ]:
bronze_erp_activo_df['activo'].value_counts(dropna=False)
# NOTE: Pese a que todos los valores de activo son representaciones
# de un booleano, no son el mismo booleano, por lo que debemos normalizar

activo
False    193
True     169
0        165
1        160
Sí       142
No       140
Name: count, dtype: int64

In [89]:
def normalizar_booleano(valor: str)->str:
    valores_truthy = ["Sí","1","True"]
    valores_falsy = ["No","0","False"]
    if valor in valores_truthy:
        return True
    elif valor in valores_falsy:
        return False
    else:
        return valor

In [90]:

bronze_erp_activo_df['activo'] = bronze_erp_activo_df['activo'].apply(normalizar_booleano)

C:\Users\Alumno\AppData\Local\Temp\ipykernel_9936\1779153276.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bronze_erp_activo_df['activo'] = bronze_erp_activo_df['activo'].apply(normalizar_booleano)


In [91]:
bronze_erp_activo_df['activo'].value_counts(dropna=False)

activo
False    498
True     471
Name: count, dtype: int64

### Duplicados

Al igual que con los nulos, las columnas críticas (claves primarias) no pueden tener dos filas representando el mismo cliente.

En caso de no tener claves primarias, usaremos nuestro criterio para saber que puede o no puede estar duplicado en nuestro dataset.

In [94]:
bronze_erp_activo_df['id_cliente'].value_counts()

id_cliente
466     2
283     2
75      2
211     2
473     2
       ..
997     1
998     1
999     1
1000    1
6       1
Name: count, Length: 950, dtype: int64

In [97]:
bronze_erp_activo_df['id_cliente'].count()

np.int64(969)

In [95]:
bronze_erp_sin_duplicados_df = bronze_erp_activo_df.drop_duplicates("id_cliente")

In [96]:
bronze_erp_sin_duplicados_df['id_cliente'].value_counts()

id_cliente
1       1
2       1
3       1
4       1
5       1
       ..
997     1
998     1
999     1
1000    1
642     1
Name: count, Length: 950, dtype: int64

In [98]:
bronze_erp_sin_duplicados_df['id_cliente'].count()

np.int64(950)

In [99]:
bronze_erp_sin_duplicados_df = bronze_erp_activo_df.drop_duplicates(["nombre","email"])

In [100]:
bronze_erp_sin_duplicados_df['id_cliente'].count()

np.int64(948)

### Tipado

Para asegurar que las operaciones entre columnas son lo más eficientes posible, y que no nos encontraremos en una situación:

"2" + "2" = "22"

Es importante comprobar que todas las columnas tienen el tipo correcto, y corregirlo en caso de que no.

In [ ]:
bronze_erp_sin_duplicados_df.info()
# NOTE: Podemos ver que está todo bien menos las columnas de fecha

<class 'pandas.core.frame.DataFrame'>
Index: 948 entries, 0 to 1010
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id_cliente        948 non-null    int64         
 1   nombre            948 non-null    object        
 2   email             948 non-null    object        
 3   edad              882 non-null    float64       
 4   ciudad            948 non-null    object        
 5   salario           948 non-null    float64       
 6   fecha_registro    948 non-null    object        
 7   categoria         948 non-null    object        
 8   telefono          948 non-null    object        
 9   activo            948 non-null    bool          
 10  puntos_fidelidad  948 non-null    float64       
 11  ultima_compra     948 non-null    object        
 12  _BronzeTimestamp  948 non-null    datetime64[us]
dtypes: bool(1), datetime64[us](1), float64(3), int64(1), object(7)
memory usage: 97.2+ K

In [106]:
bronze_erp_sin_duplicados_df['ultima_compra'] = pd.to_datetime(bronze_erp_sin_duplicados_df['ultima_compra'])

C:\Users\Alumno\AppData\Local\Temp\ipykernel_9936\4279201018.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bronze_erp_sin_duplicados_df['ultima_compra'] = pd.to_datetime(bronze_erp_sin_duplicados_df['ultima_compra'])


In [108]:
bronze_erp_sin_duplicados_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 948 entries, 0 to 1010
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id_cliente        948 non-null    int64         
 1   nombre            948 non-null    object        
 2   email             948 non-null    object        
 3   edad              882 non-null    float64       
 4   ciudad            948 non-null    object        
 5   salario           948 non-null    float64       
 6   fecha_registro    948 non-null    object        
 7   categoria         948 non-null    object        
 8   telefono          948 non-null    object        
 9   activo            948 non-null    bool          
 10  puntos_fidelidad  948 non-null    float64       
 11  ultima_compra     948 non-null    datetime64[ns]
 12  _BronzeTimestamp  948 non-null    datetime64[us]
dtypes: bool(1), datetime64[ns](1), datetime64[us](1), float64(3), int64(1), object(6)
me

In [116]:
bronze_erp_sin_duplicados_df['fecha_registro'] = pd.to_datetime(bronze_erp_sin_duplicados_df['fecha_registro'])

C:\Users\Alumno\AppData\Local\Temp\ipykernel_9936\3497994002.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bronze_erp_sin_duplicados_df['fecha_registro'] = pd.to_datetime(bronze_erp_sin_duplicados_df['fecha_registro'])


In [117]:
bronze_erp_sin_duplicados_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 948 entries, 0 to 1010
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id_cliente        948 non-null    int64         
 1   nombre            948 non-null    object        
 2   email             948 non-null    object        
 3   edad              882 non-null    float64       
 4   ciudad            948 non-null    object        
 5   salario           948 non-null    float64       
 6   fecha_registro    948 non-null    datetime64[ns]
 7   categoria         948 non-null    object        
 8   telefono          948 non-null    object        
 9   activo            948 non-null    bool          
 10  puntos_fidelidad  948 non-null    float64       
 11  ultima_compra     948 non-null    datetime64[ns]
 12  _BronzeTimestamp  948 non-null    datetime64[us]
dtypes: bool(1), datetime64[ns](2), datetime64[us](1), float64(3), int64(1), object(5)
me

### Guardar

In [109]:
# Crear carpeta
os.mkdir(silver_path)

In [119]:
bronze_erp_sin_duplicados_df.to_parquet(silver_file_path)